In [8]:
using LinearAlgebra
using TensorKit
using KrylovKit
using JSON
using HDF5, JLD
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("kagome_load_tensor.jl")
include("kagome_CTMRG.jl")
include("kagome_model.jl")
include("kagome_IPESS.jl")
include("mps_algorithms\\ITEBD_algorithms.jl")
include("mps_algorithms\\TransfOp_decomposition.jl")
include("mps_algorithms\\PUMPS_algorithms.jl")
include("mps_algorithms\\ES_preliminary.jl")



D=8;
chi=10;
W=10;
N=10;
kset=0:9;
EH_n=3;#number of entanglement spectrum per k point
Dtrun_method="svds";
Dtrun_init=200;
Dtrun_max=200;
unitcell_size=2;

J1=0.80902;
J2=0;
J3=0;
Jchi=0;
Jtrip=0.5878;
save_mat_tensors=false;
parameters=Dict([("J1", J1), ("J2", J2), ("J3", J3), ("Jchi", Jchi), ("Jtrip", Jtrip)]);



In [2]:

    
    
    multi_threads=true;if Threads.nthreads()==1; multi_threads=false; end
    println("number of threads: "*string(Threads.nthreads()));flush(stdout);
    
    CTM_conv_tol=1e-6;
    trun_tol=1e-8;
    group_size=Int(round((10^8)/(chi*chi*W*W*D)));
    
    mpo_type="OO";#"O_O" or "OO", in my test "OO" is faster for large bond dimension
    
    pow=Int((N-2)/2);
    

    
    A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);
    
    filenm="LS_D_"*string(D)*"_chi_40.json"
    json_dict=read_json_state(filenm);
    
    bond_tensor,triangle_tensor=construct_su2_PG_IPESS(json_dict,A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb);
    
    PEPS_tensor=bond_tensor;
    @tensor PEPS_tensor[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];
    A_unfused=PEPS_tensor;
    
    U_phy=unitary(fuse(space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7)), space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7));
    @tensor A_fused[:] :=PEPS_tensor[-1,-2,-3,-4,1,2,3]*U_phy[-5,1,2,3];
    
    
    
    CTM,U_L,U_D,U_R,U_U=try_CTM(D,chi,parameters, CTM_conv_tol, U_phy, A_unfused, A_fused);
    


number of threads: 1


load CTM from saved data directly


(Dict{String, Vector{Any}}("Cset" => [TensorMap((Rep[SU₂](0=>3, 1/2=>2, 1=>1)' ⊗ Rep[SU₂](0=>3, 1/2=>2, 1=>1)) ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()):
* Data for fusiontree FusionTree{Irrep[SU₂]}((1, 1), 0, (true, false), ()) ← FusionTree{Irrep[SU₂]}((), 0, (), ()):
 0.012797537533271963 - 0.4914201887144059im
* Data for fusiontree FusionTree{Irrep[SU₂]}((1/2, 1/2), 0, (true, false), ()) ← FusionTree{Irrep[SU₂]}((), 0, (), ()):
    0.260481602516181 + 0.42167595083091im      …  0.003669002618563458 + 0.021023005331309554im
 0.006717927351874831 - 0.007950985307522637im      0.06867358251392658 - 0.4557591511763716im
* Data for fusiontree FusionTree{Irrep[SU₂]}((0, 0), 0, (true, false), ()) ← FusionTree{Irrep[SU₂]}((), 0, (), ()):
  -0.11449052209928863 - 0.4354540224831186im    …  -0.007986636769978503 + 0.0006678172420589823im
 -0.010144087380140495 - 0.006416572313207222im       0.00692127847436566 + 0.009787472435254454im
 -0.0048345

In [10]:
    Ag,O1,O2=try_ITEBD(D,chi,W,CTM,U_L,U_R,unitcell_size);

    if save_mat_tensors
        ES_filenm="Ag_mpo_tensors_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*".mat";
        matwrite(ES_filenm, Dict(
            "Ag" => convert(Array,Ag),
            "O1" => convert(Array,O1),
            "O2" => convert(Array,O2),
            "C1" => convert(Array,CTM["Cset"][1]),
            "C2" => convert(Array,CTM["Cset"][2]),
            "C3" => convert(Array,CTM["Cset"][3]),
            "C4" => convert(Array,CTM["Cset"][4]),
            "T1" => convert(Array,CTM["Tset"][1]),
            "T2" => convert(Array,CTM["Tset"][2]),
            "T3" => convert(Array,CTM["Tset"][3]),
            "T4" => convert(Array,CTM["Tset"][4])
        ); compress = false)
    end
    
    


No itebd ground state found from saved data, now do itebd


2

2

2

"Warn: dominant transfer matrix eigenvalue close to degenerate: 0.9918301617534659"

...

"Warn: dominant transfer matrix eigenvalue close to degenerate: 0.9999975105076264"

"Warn: dominant transfer matrix eigenvalue close to degenerate: 0.9999975105076202"

2

  2.971005 seconds (6.93 M allocations: 485.862 MiB, 7.80% gc time, 67.53% compilation time)
E=0.053384675217268716 - 0.00047073397354783407im, ov=0.28500104436196133
  1.127951 seconds (2.73 M allocations: 262.236 MiB, 6.08% gc time)
E=0.059503633673618515 - 0.0005222270954929325im, ov=0.8928086005047482
  1.107645 seconds (2.70 M allocations: 255.494 MiB)
E=0.05933661636316702 - 0.0005053262229790654im, ov=0.9919102441628789
  1.089866 seconds (2.70 M allocations: 251.708 MiB, 9.48% gc time)
E=0.05939959057190318 - 0.0005147849635697983im, ov=0.9997285489150267
  1.309731 seconds (2.70 M allocations: 251.708 MiB, 9.02% gc time)
E=0.05941215636168902 - 0.0005178958985826507im, ov=0.9999570676241487
  0.958385 seconds (2.70 M allocations: 251.708 MiB)
E=0.059415026701715105 - 0.0005191799973084568im, ov=0.9999733054116714
  1.152551 seconds (2.70 M allocations: 251.708 MiB, 9.95% gc time)
E=0.059415865290805996 - 0.0005200257018207362im, ov=0.9999743047025863
  1.194666 seconds (2.70 M

┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 2`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\arnoldi.jl:176
┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 2`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\arnoldi.jl:176
┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 2`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\arnoldi.jl:176
┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues 

In [11]:



    space_AOA=fuse(space(Ag,1)'⊗space(O2,1)'⊗space(O1,1)⊗ space(Ag,1));
    space_AA=fuse(space(Ag,1)'⊗ space(Ag,1));
    
    AOA_sec=collect(sectors(space_AOA))
    AA_sec=collect(sectors(space_AA))
    
    @tensor OO[:]:=O1[-2,-3,-5,1]*O2[-1,1,-4,-6];
    U_fuse_chichi=unitary(fuse(space(OO,1)⊗ space(OO,2)),space(OO,1)⊗ space(OO,2));
    @tensor OO[:]:=U_fuse_chichi[-1,1,2]*OO[1,2,-2,3,4,-4]*U_fuse_chichi'[3,4,-3];
    
    
    
    
    #normalize the MPO
    euL_set,_,_,_,_=FLR_eig(Ag,OO,20,space_AOA,AOA_sec);
    norm_coe=maximum(abs.(group_numbers(euL_set)));
    OO=OO/norm_coe;
    O1=O1/norm_coe;
    
    


TensorMap((Rep[SU₂](0=>3, 1/2=>2, 1=>1) ⊗ Rep[SU₂](0=>6, 1/2=>8, 1=>7, 3/2=>4, 2=>1)' ⊗ Rep[SU₂](0=>3, 1/2=>2, 1=>1)' ⊗ Rep[SU₂](0=>6, 1/2=>8, 1=>7, 3/2=>4, 2=>1)) ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()):
* Data for fusiontree FusionTree{Irrep[SU₂]}((1/2, 3/2, 1/2, 3/2), 0, (false, true, true, false), (2, 3/2)) ← FusionTree{Irrep[SU₂]}((), 0, (), ()):
[:, :, 1, 1] =
    0.2912552535519725 + 0.0071697995433758im     …   -0.10135342364426432 - 0.3369182222119902im
 -0.004978858932534954 - 0.0009738111564148274im     -0.018428536107093073 - 0.06601982361885679im

[:, :, 2, 1] =
 0.03195327241428989 - 0.024482863255988415im  …   -0.1160636820184243 + 0.07856363968464768im
 0.14946488634299107 - 0.04874345229659528im      -0.13127396825861795 - 0.2685125581309572im

[:, :, 1, 2] =
  0.23746992560391092 - 0.24043491923360483im  …    -1.1525308606080398 - 0.9265215098535985im
 0.016196832613915618 - 0.0677566225356652im      -0.0590491394593025

In [15]:
    
    
    
    euR_set,evL_set,evR_set,SPIN_eig_set=TransfOp_decom(Ag,OO,space_AOA,AOA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"eigenvalue_FLR");
    # println(euR_set)
    
    eur_set,evl_set,evr_set,spin_eig_set=TransfOp_decom(Ag,OO,space_AA,AA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"eigenvalue_GLR");
    # println(eur_set)
    
    S_set,U_set,Vh_set,SPIN_svd_set=TransfOp_decom(Ag,OO,space_AOA,AOA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"svd_FLR");
    # println(S_set)
    
    s_set,u_set,vh_set,spin_svd_set=TransfOp_decom(Ag,OO,space_AA,AA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"svd_GLR");
    # println(s_set)
    


Number of eigenvalues obtained are not enough, use smaller tol
minimal singular value in this sector is quite small, skip checking number of converged values


Number of eigenvalues obtained are not enough, use smaller tol
minimal singular value in this sector is quite small, skip checking number of converged values


Number of eigenvalues obtained are not enough, use smaller tol
minimal singular value in this sector is quite small, skip checking number of converged values


Number of eigenvalues obtained are not enough, use smaller tol
minimal singular value in this sector is quite small, skip checking number of converged values


FLR_eig, DTrun=200, SU2 reduced to 110, minimal eigenvalue: 0.00024035716051159375


...

Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol
Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol
minimal singular value in this sector is quite small, skip checking number of converged values


Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol
minimal singular value in this sector is quite small, skip checking number of converged values


GLR_svd, Dtrun=200, SU2 reduced to 42, minimal eigenvalue: 1.9753146199482065e-94


┌ Warning: Invariant subspace of dimension 17 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 30`); setting `howmany = 17`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\arnoldi.jl:176
┌ Warning: Invariant subspace of dimension 17 (up to requested tolerance `tol = 1.0e-14`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 30`); setting `howmany = 17`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\arnoldi.jl:176
┌ Warning: Invariant subspace of dimension 17 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 30`); setting `howmany = 17`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\arnoldi.jl:176
┌ Warning: Invariant subspace of dimension 17 (up to requested tolerance `tol = 1.0e-14`), which is smaller than the number of requested ei

(Any[[1.4818772316336466, 1.169814443881915, 0.2667668402598024, 0.2667668401575254, 0.14357254152719778, 0.143572541461742, 0.005690204481113771, 0.003621141542136125, 1.9571267051816675e-6, 1.9571267050446667e-6, 2.2661186634437147e-7, 1.2893198447273184e-7], [0.29006519076876847, 0.2900651907196514, 0.0679303066782586, 0.06793030662475166, 0.058779219860183786, 0.05877921981397973, 0.00045050363539106305, 0.0004505036353035781, 5.136073703288616e-5, 5.136073700525042e-5, 1.5750701702072473e-6, 1.5750701661954962e-6, 1.3628879432196614e-6, 1.36288793618455e-6, 2.9051969818019085e-7, 2.9051969801761627e-7], [0.003160766600638362, 0.0021197522717305867, 0.0003959240508477524, 0.00039592405061111526, 0.00016449347166109447, 0.00016449347163229603], [3.250519409091962e-7, 3.250519409091962e-7], Float64[]], Any[Any[TensorMap((Rep[SU₂](0=>3, 1/2=>2, 1=>1) ⊗ Rep[SU₂](0=>3, 1/2=>2, 1=>1)') ← ProductSpace(Rep[SU₂](0=>1))):
* Data for fusiontree FusionTree{Irrep[SU₂]}((1/2, 1/2), 0, (false, tr

In [16]:
    
    check_truncated_decomp_error=false;
    
    if mpo_type=="O_O"
        OO_transform=true;
    elseif mpo_type=="OO"
        OO_transform=false;
    end
    
    euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined=combine_singlespin_sector(euR_set,evL_set,evR_set,SPIN_eig_set,true);
    euR_set_grouped,evL_set_grouped,evR_set_grouped,SPIN_eig_set_grouped,DTrun_FLR_eig=group_singlespin_sector(group_size,euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined,OO_transform,U_fuse_chichi)
    println("group information:");flush(stdout);
    println(DTrun_FLR_eig);flush(stdout);
    
    eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined=combine_singlespin_sector(eur_set,evl_set,evr_set,spin_eig_set,true)
    eur_set_grouped,evl_set_grouped,evr_set_grouped,spin_eig_set_grouped,Dtrun_GLR_eig=group_singlespin_sector(group_size,eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined,false,[])
    println("group information:");flush(stdout);
    println(Dtrun_GLR_eig);flush(stdout);
    
    
    S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined=combine_singlespin_sector(S_set,Vh_set,U_set,SPIN_svd_set,false)
    S_set_grouped,Vh_set_grouped,U_set_grouped,SPIN_svd_set_grouped,DTrun_FLR_svd=group_singlespin_sector(group_size,S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined,OO_transform,U_fuse_chichi)
    println("group information:");flush(stdout);
    println(DTrun_FLR_svd);flush(stdout);
    
    s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined=combine_singlespin_sector(s_set,vh_set,u_set,spin_svd_set,false)
    s_set_grouped,vh_set_grouped,u_set_grouped,spin_svd_set_grouped,Dtrun_GLR_svd=group_singlespin_sector(group_size,s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined,false,[])
    println("group information:");flush(stdout);
    println(Dtrun_GLR_svd);flush(stdout);
    
    
    
    
    ES_sectors=[0,1/2,1,3/2,2,5/2];
    
    #kset=0:0
    Eset=[];
    Trun_err=0;
    DTrun=0;
    println("calculate ES for N="*string(N));
    println("kset="*string(kset));flush(stdout);
    pow=round((N-2)/2);
    
    
    
    if Dtrun_method=="eigs"
        DTrun=length(group_numbers(SPIN_eig_set));
        println("DTrun="*string(DTrun));
    
        euRs=abs.(group_numbers(euR_set));
        Trun_err=(minimum(euRs)/maximum(euRs))^pow;
    
    
        euR_pow=deepcopy(euR_set_grouped);
        for ca=1:length(euR_pow)
            for cb=1:length(euR_pow[ca])
                euR_pow[ca][cb]=euR_pow[ca][cb]^Int(pow);
            end
        end
    
        kset,Eset=solve_ITEBD_excitation_TrunTransOp_iterative(Ag,O1,O2,OO,EH_n,N,kset,ES_sectors,pow,evR_set_grouped,euR_pow,evL_set_grouped,SPIN_eig_set_grouped,DTrun_FLR_eig,mpo_type,multi_threads)
    
    elseif Dtrun_method=="svds"
        DTrun=length(group_numbers(SPIN_svd_set));
        println("DTrun="*string(DTrun));
    
        Ss=abs.(group_numbers(S_set));
        Trun_err=(minimum(Ss)/maximum(Ss));
    
        kset,Eset=solve_ITEBD_excitation_TrunTransOp_iterative(Ag,O1,O2,OO,EH_n,N,kset,ES_sectors,pow,U_set_grouped,S_set_grouped,Vh_set_grouped,SPIN_svd_set_grouped,DTrun_FLR_svd,mpo_type,multi_threads)
    end
    
    ES_filenm="ES_"*Dtrun_method*"_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*"_N"*string(N)*"_kset"*string(kset[1])*"to"*string(kset[end])*".mat";
    matwrite(ES_filenm, Dict(
        "kset" => convert(Vector,kset),
        "ES_sectors" => ES_sectors,
        "Eset" => Eset,
        "Trun_err"=>Trun_err,
        "DTrun"=>DTrun
    ); compress = false)

group information:


Any[UnitRange{Int64}[1:42], UnitRange{Int64}[1:48], UnitRange{Int64}[1:18], UnitRange{Int64}[1:2], ComplexF64[], ComplexF64[], ComplexF64[], ComplexF64[], ComplexF64[]]


group information:


Any[UnitRange{Int64}[1:12], UnitRange{Int64}[1:16], UnitRange{Int64}[1:6], UnitRange{Int64}[1:2], ComplexF64[]]


group information:


...

Any[UnitRange{Int64}[1:12], UnitRange{Int64}[1:16], UnitRange{Int64}[1:6], UnitRange{Int64}[1:2], Float64[]]


calculate ES for N=10
kset=0:9


DTrun=109
sector k=0, spin=0.0:


934.393716 seconds (421.57 M allocations: 48.022 GiB, 3.90% gc time, 23.18% compilation time)
ComplexF64[1.52483346907317 + 0.2659604992769288im, 0.7388963185547587 + 0.1488772233397019im, 0.6209176760377635 + 0.12524287991900582im, 0.5737278410087269 + 0.11267718645092605im]
 


sector k=0, spin=0.5:
